In [214]:
import pypandoc
from bs4 import BeautifulSoup
import os
import json

In [215]:
def card_exists(dictionary, link, content):
    for value in dictionary.values():
        for item in value:
            if 'link' in item and item['link'] == link:
                return True
            if 'content' in item and item['content'] == content:
                return True
    return False

def converttoHTML(filepath):
    allHtml = {}
    try:
        output = pypandoc.convert_file(filepath, 'html')
        filepath_to_save = filepath.split(
            "NSDA-Case-Files/")[1].split(".docx")[0] + ".html"

        with open('test.html', 'w') as fp:
            fp.write(output)
            file_name = filepath.split(".docx")[0] + ".html"
            os.rename('test.html', file_name)

        num_of_cards = 1
        with open(file_name) as fp:
            soup = BeautifulSoup(fp, "lxml")
            all_h4_tags = soup.find_all('h4')
            all_strong_tags = soup.find_all('strong')
            if len(all_h4_tags) > len(all_strong_tags) or len(all_h4_tags) > 5:
                all_card_tags = all_h4_tags
            else:
                all_card_tags = all_strong_tags

            for element in all_card_tags:
                try:
                    text_with_link = element.find_next("p")
                    link = ""

                    for i in range(3):
                        if "http" in text_with_link.text:
                            link = next((word for word in text_with_link.text.split(
                                " ") if "http" in word), None)
                            break
                        text_with_link = text_with_link.find_next("p")

                    card_content = text_with_link.find_next("p").text

                    if len(card_content.split(" ")) >= 70:
                        if card_exists(allHtml, link, card_content):
                            continue
                        
                        allHtml[str(num_of_cards)] = [{"file": str(filepath_to_save), "link": str(
                            link), "content": str(card_content)}]
                        num_of_cards += 1
                    
                    link = ""
                except AttributeError as e:
                    # print("a card was skipped because " + str(e))
                    pass
        return allHtml
    except Exception as e:
        # print("a card was skipped because " + str(e))
        return allHtml

In [216]:
def traverse_directory(directory_path, output_file):
    all_html = {}
    for root, dirs, files in os.walk(directory_path):
        for file in files:
            if file.endswith(".docx"):
                file_path = os.path.join(root, file)
                file_link = "file://" + file_path
                file_name = os.path.splitext(file)[0]
                print("starting: " + file_path)

                cards = converttoHTML(file_path)
                all_html[file_name] = cards

    with open(output_file, 'w') as f:
        json.dump(all_html, f)

    return all_html

In [217]:
traverse_directory("/home/skeptrune/Downloads/NSDA-Case-Files/", "./case-data.json")

starting: /home/skeptrune/Downloads/NSDA-Case-Files/test/Advantage Counterplans - Northwestern 2013 6WeekSeniors.docx


{'Advantage Counterplans - Northwestern 2013 6WeekSeniors': {'1': [{'file': 'test/Advantage Counterplans - Northwestern 2013 6WeekSeniors.html',
    'link': 'http://www.greenpeace.org/international/en/press/releases/soya-traders-extend-moratorium/)',
    'content': '"This is the sort of industry initiative we need to stop the destruction of the Amazon and help to prevent runaway climate change. It was only possible because companies worldwide demanded it, knowing that their customers would not want to buy products linked to Amazon rainforest destruction,"said Paulo Adario, Greenpeace Amazon Campaign director.¶ "Now all eyes are on President Lula to demonstrate his government’s leadership by increasing efforts to govern the Amazon and to stop deforestation across all sectors.” ¶ The soya moratorium extension was announced at a press conference in Brasília attended by Brazil’s Environment Minister Carlos Minc and the Soya Working Group.¶ Soya cultivation in the Amazon increased dramatica